In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import dask
from dask.diagnostics import ProgressBar
import os

In [3]:
tmp_data_path = '../MA_data/data/tmp'
data_path = '../MA_data/data'

s_year = 1997
e_year = 2020

In [4]:
sdc_tnic = pd.read_pickle(tmp_data_path+f"/sdc_tnic_{s_year}_{e_year}")

In [5]:
with open(tmp_data_path+f"/tnic_info_3_pairs_{s_year-1}_{e_year-1}", 'rb') as f:
    gvkey_lsts, key_ind_maps , ind_key_maps = pickle.load(f)

In [30]:
tnic2 = pd.read_pickle(tmp_data_path+f"/tnic2_{s_year-1}_{e_year-1}") # use tnic 3

FileNotFoundError: [Errno 2] No such file or directory: '../MA_data/data/tmp/tnic2_1996_2020'

In [7]:
A_freq = pd.DataFrame(sdc_tnic.AGVKEY.value_counts()).reset_index(drop=False)
A_freq = A_freq[A_freq.AGVKEY >= 5]
A_freq.columns = ["GVKEY", "freq"]
print(f"totally {A_freq.shape[0]} numbers of frequent Acquirers")

totally 511 numbers of frequent Acquirers


In [32]:
fv_all = pd.read_pickle(tmp_data_path+f'/fv_raw_{s_year-1}_{e_year}.pickle')

# Q: Can we use TNIC3 instead of TNIC2? (save computation)
- No, since TNIC3 has limited graininess. A lot of firms has no peers at all...

# Q: Top peer... who can make influence?
- Those top peers that acquirer something
    - no need to be frequent acquirer

# Q: put how much financial vars for `focal frequent acquirer` each batch?
- all between min year and max year

# Q: Top K peers?
- trying top 10....
    - 

In [10]:
A_freq[:11]

,GVKEY,freq
0,5047,101
1,2176,49
2,177088,41
3,11856,41
4,6268,39
5,8007,30
6,3243,29
7,9899,28
8,6266,28
9,114628,26


In [11]:
focal_gvkey = "177088"

for focal_year in range(s_year-1, e_year):
    sdc_lst = []
    with open(tmp_data_path+f"/a5_top_peers_tnic3_{focal_year}", 'rb') as f:
        top_peers = pickle.load(f)
    try:
        top_peers = top_peers[focal_gvkey]
        selected_sdc_tnic = sdc_tnic[ (sdc_tnic['AGVKEY'].isin(top_peers + [focal_gvkey])) & (sdc_tnic.YEAR == focal_year+1) ]
        sdc_lst.append(selected_sdc_tnic)
    except:
        pass



focal_gvkey_sdc = pd.concat(sdc_lst, axis=1)
focal_gvkey_sdc

,ACU,ASIC2,ABL,ANL,APUBC,AUP,AUPSIC,AUPBL,AUPNAMES,AUPPUB,...,TS_PERMNO,TP_PERMNO,AS_GVKEY,AP_GVKEY,TS_GVKEY,TP_GVKEY,GVKEY_STATUS,MA_TYPE,AGVKEY,TGVKEY
193164,58625P,8049,"MedRisk LLC, based in King\nOf Prussia, Pennsy...",MedRisk LLC,Sub.,14316J,6799\n6726\n6282\n6159,"The Carlyle Group Inc,\nlocated in Washington ...",The Carlyle Group Inc,Public,...,NaN,93035,NaN,189459,NaN,174236,9,PP,189459,174236
193298,09260D,6799/6726/6282,"Blackstone Group Inc,\nlocated in New York Cit...",Blackstone Group Inc,Public,09260D,6799\n6726\n6282,"Blackstone Group Inc,\nlocated in New York Cit...",Blackstone Group Inc,Public,...,NaN,85913,177088,177088,NaN,28930,3,SP,177088,28930
193467,09252C,6798,"Blackstone Real Estate\nPartners LP, located i...",Blackstone Real Estate Partners LP,Sub.,09260D,6799\n6726\n6282,"Blackstone Group Inc,\nlocated in New York Cit...",Blackstone Group Inc,Public,...,NaN,19583,NaN,177088,NaN,29804,9,PP,177088,29804
194613,354613,6282/6289/6141/6733/6722,"Franklin Resources Inc,\nlocated in San Mateo,...",Franklin Resources Inc,Public,354613,6282\n6289\n6141\n6733\n6722,"Franklin Resources Inc,\nlocated in San Mateo,...",Franklin Resources Inc,Public,...,65330,65330,4885,4885,6653,6653,1,SS,4885,6653
195441,48251W,6799,"KKR & Co Inc, located in New\nYork City, New Y...",KKR & Co Inc,Public,48251W,6799,"KKR & Co Inc, located in New\nYork City, New Y...",KKR & Co Inc,Public,...,NaN,70703,179741,179741,NaN,13579,3,SP,179741,13579
196277,19247A,6282/6722/6726/6799,"Cohen & Steers Inc, located\nin New York City,...",Cohen & Steers Inc,Public,19247A,6282\n6722\n6726\n6799,"Cohen & Steers Inc, located\nin New York City,...",Cohen & Steers Inc,Public,...,80412,80412,266214,266214,30067,30067,1,SS,266214,30067
197795,48251W,6799,"KKR & Co Inc, located in New\nYork City, New Y...",KKR & Co Inc,Public,48251W,6799,"KKR & Co Inc, located in New\nYork City, New Y...",KKR & Co Inc,Public,...,NaN,69032,179741,179741,NaN,12124,3,SP,179741,12124
200769,86198E,8748/8742/7375/7374,CorroHealth Inc is a\nprovider of technical\nc...,CorroHealth Inc,Sub.,14316J,6799\n6726\n6282\n6159,"The Carlyle Group Inc,\nlocated in Washington ...",The Carlyle Group Inc,Public,...,NaN,90925,NaN,189459,NaN,165264,9,PP,189459,165264
202000,3H2320,6726/6799,"Carlyle Partners VII LP,\nlocated in Washingto...",Carlyle Partners VII LP,Sub.,14316J,6799\n6726\n6282\n6159,"The Carlyle Group Inc,\nlocated in Washington ...",The Carlyle Group Inc,Public,...,NaN,69032,NaN,189459,NaN,12124,9,PP,189459,12124
202682,43120E,6411/6321/6331/6351,"Hilb Group LLC, located in\nRichmond, Virginia...",Hilb Group LLC,Sub.,14316J,6799\n6726\n6282\n6159,"The Carlyle Group Inc,\nlocated in Washington ...",The Carlyle Group Inc,Public,...,NaN,38093,NaN,189459,NaN,4973,9,PP,189459,4973


# Obtain sub dataset

In [11]:
A_freq[:10]

,GVKEY,freq
0,5047,101
1,2176,49
2,177088,41
3,11856,41
4,6268,39
5,8007,30
6,3243,29
7,9899,28
8,6266,28
9,6008,26


In [27]:

focal_gvkey = "5047"

sdc_lst = []
for focal_year in range(s_year-1, e_year):
    with open(tmp_data_path+f"/a5_top_10_peers_tnic3_{focal_year}", 'rb') as f: # must use tnic2!
        top_peers = pickle.load(f)
        #print(top_peers)
    try:
        top_peers = top_peers[focal_gvkey] # return a sub dataframe
        top_peers_lst = top_peers.gvkey2.tolist()
        
        selected_sdc_tnic = sdc_tnic[ (sdc_tnic['AGVKEY'].isin(top_peers_lst + [focal_gvkey])) & (sdc_tnic.YEAR == focal_year+1) ]
        print(len(top_peers_lst), top_peers_lst, selected_sdc_tnic.shape[0])
        sdc_lst.append(selected_sdc_tnic)
    except:
        pass


focal_gvkey_sdc = pd.concat(sdc_lst, ignore_index=True)

1 ['8972'] 14
3 ['10519', '10983', '12581'] 10
8 ['1447', '1831', '3243', '3282', '5087', '9380', '10983', '62689'] 15
5 ['1447', '1831', '9380', '10983', '62689'] 9
5 ['6331', '9380', '10983', '14793', '62689'] 16
7 ['1794', '4058', '5860', '6331', '10519', '10983', '14793'] 12
8 ['1487', '1598', '1773', '2049', '5860', '7585', '10519', '28034'] 11
0 [] 5
2 ['5860', '25405'] 4
3 ['4321', '10983', '25405'] 7
5 ['4058', '4321', '6216', '8358', '10983'] 7
2 ['4321', '6216'] 4
2 ['5860', '10983'] 4
1 ['10983'] 2
1 ['10983'] 3
3 ['3735', '10983', '11580'] 4
1 ['10983'] 2
3 ['1598', '10983', '11580'] 3
0 [] 1
0 [] 1
0 [] 1
0 [] 2
0 [] 0
0 [] 0


In [33]:
fv_all.columns

Index(['gvkey', 'year', 'at', 'sale', 'ch', 'rdip', 'm2b', 'lev', 'roa', 'ppe',
       'cash2asset', 'cash2sale', 'sale2asset', 'cr', 'gsi', 'de', 'roe',
       'd_sale', 'd_at'],
      dtype='object')